# Google Vertex AI Vector Search

This notebook shows how to use functionality related to the `Google Cloud Vertex AI Vector Search` vector database.

> [Google Vertex AI Vector Search](https://cloud.google.com/vertex-ai/docs/vector-search/overview), formerly known as Vertex AI Matching Engine, provides the industry's leading high-scale low latency vector database. These vector databases are commonly referred to as vector similarity-matching or an approximate nearest neighbor (ANN) service.

**Note**: Langchain API expects an endpoint and deployed index already created.Index creation time can take upto one hour. <br>
> To see how to create an index refer to the section [Create Index and deploy it to an Endpoint](#create-index-and-deploy-it-to-an-endpoint)<br>
> If you already have an index deployed , skip to [Create VectorStore from texts](#create-vector-store-from-texts)

## Create Index and deploy it to an Endpoint
- This section demonstrates creating a new index and deploying it to an endpoint

In [48]:
#TODO : Set values as per your requirements 
# Project and Storage Constants
PROJECT_ID = "<my_project_id>"
REGION = "<my_region>"
BUCKET = "<my_gcs_bucket>"
BUCKET_URI = f"gs://{BUCKET}"

# The number of dimensions for the textembedding-gecko@003.
# If other embedder is used, the dimensions would probably need to change.
DIMENSIONS = 768

# Index Constants
DISPLAY_NAME = "<my_matching_engine_index_id>"
DEPLOYED_INDEX_ID = "<my_matching_engine_endpoint_id>"

In [42]:
# Create a bucket.
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

Traceback (most recent call last):
  File "/usr/local/share/google-cloud-sdk/bin/bootstrapping/gsutil.py", line 16, in <module>
    from googlecloudsdk.calliope import exceptions
ModuleNotFoundError: No module named 'googlecloudsdk'


### Use [VertexAIEmbeddings](https://python.langchain.com/docs/integrations/text_embedding/google_vertex_ai_palm/) as the embeddings model

In [43]:
from langchain_google_vertexai import VertexAIEmbeddings
from google.cloud import aiplatform

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

In [44]:
embedding_model = VertexAIEmbeddings(model_name="textembedding-gecko@003")

### Create an empty Index 

**Note :** While creating an index you should specify an "index_update_method" from either a "BATCH_UPDATE" or "STREAM_UPDATE"
> A batch index is for when you want to update your index in a batch, with data which has been stored over a set amount of time, like systems which are processed weekly or monthly. A streaming index is when you want index data to be updated as new data is added to your datastore, for instance, if you have a bookstore and want to show new inventory online as soon as possible. Which type you choose is important, since setup and requirements are different.

Refer [Official Documentation](https://cloud.google.com/vertex-ai/docs/vector-search/create-manage-index#create-index-batch) for more details on configuring indexes


In [49]:
# NOTE : This operation can take upto 30 seconds
my_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
    display_name=DISPLAY_NAME,
    dimensions=DIMENSIONS,
    approximate_neighbors_count=150,
    distance_measure_type="DOT_PRODUCT_DISTANCE",
    index_update_method="STREAM_UPDATE",  # allowed values BATCH_UPDATE , STREAM_UPDATE
)

Creating MatchingEngineIndex
Create MatchingEngineIndex backing LRO: projects/543079149601/locations/us-central1/indexes/6712989078549168128/operations/8589295810104000512
MatchingEngineIndex created. Resource name: projects/543079149601/locations/us-central1/indexes/6712989078549168128
To use this MatchingEngineIndex in another session:
index = aiplatform.MatchingEngineIndex('projects/543079149601/locations/us-central1/indexes/6712989078549168128')


### Create an Endpoint

In [50]:
#Create an endpoint
my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
    display_name=f"{DISPLAY_NAME}-endpoint"
    , public_endpoint_enabled=True
)

Creating MatchingEngineIndexEndpoint
Create MatchingEngineIndexEndpoint backing LRO: projects/543079149601/locations/us-central1/indexEndpoints/8112764137731260416/operations/6245172204057657344
MatchingEngineIndexEndpoint created. Resource name: projects/543079149601/locations/us-central1/indexEndpoints/8112764137731260416
To use this MatchingEngineIndexEndpoint in another session:
index_endpoint = aiplatform.MatchingEngineIndexEndpoint('projects/543079149601/locations/us-central1/indexEndpoints/8112764137731260416')


### Deploy Index to the Endpoint

In [51]:
# NOTE : This operation can take upto 20 minutes
my_index_endpoint = my_index_endpoint.deploy_index(
    index=my_index
    , deployed_index_id=DEPLOYED_INDEX_ID
)

my_index_endpoint.deployed_indexes

Deploying index MatchingEngineIndexEndpoint index_endpoint: projects/543079149601/locations/us-central1/indexEndpoints/8112764137731260416
Deploy index MatchingEngineIndexEndpoint index_endpoint backing LRO: projects/543079149601/locations/us-central1/indexEndpoints/8112764137731260416/operations/1543414193082859520
MatchingEngineIndexEndpoint index_endpoint Deployed index. Resource name: projects/543079149601/locations/us-central1/indexEndpoints/8112764137731260416


[id: "langchain_index_endpoint"
index: "projects/543079149601/locations/us-central1/indexes/6712989078549168128"
create_time {
  seconds: 1712908447
  nanos: 644030000
}
index_sync_time {
  seconds: 1712909387
  nanos: 61605000
}
automatic_resources {
  min_replica_count: 2
  max_replica_count: 2
}
deployment_group: "default"
]

## Create Vector Store from texts

In [52]:
from langchain_google_vertexai import VectorSearchVectorStore , VectorSearchVectorStoreDatastore

### Create simple vectorstore ( without filters)

In [54]:
#Input texts
texts = [
    "The cat sat on",
    "the mat.",
    "I like to",
    "eat pizza for",
    "dinner.",
    "The sun sets",
    "in the west.",
]

# Create a Vector Store
vector_store = VectorSearchVectorStore.from_components(
    project_id=PROJECT_ID,
    region=REGION,
    gcs_bucket_name=BUCKET,
    index_id=my_index.name,
    endpoint_id=my_index_endpoint.name,
    embedding=embedding_model,
    stream_update=True,
)

#Add vectors and mapped text chunks to your vectore store
vector_store.add_texts(texts=texts)

Upserting datapoints MatchingEngineIndex index: projects/543079149601/locations/us-central1/indexes/6712989078549168128
MatchingEngineIndex index Upserted datapoints. Resource name: projects/543079149601/locations/us-central1/indexes/6712989078549168128


['400a955d-41d5-4399-ad1b-aad5970f65dc',
 '2c54d008-c545-44e8-92b2-ddacc251f57b',
 '2663cf34-484f-4392-b622-ae997312ea91',
 'e8cb03b9-ea82-44da-939e-f9e59f380c0a',
 '8e59a999-8050-4e99-95ce-835953812a9e',
 '7b18cd9e-0697-4833-87b1-feeaae276368',
 'bd200df2-1494-48e1-995d-b4cb536362ac']

### OPTIONAL : You can also create vectore and store chunks in a Datastore 

In [ ]:
# NOTE : This operation can take upto 20 mins
vector_store = VectorSearchVectorStoreDatastore.from_components(
    project_id=PROJECT_ID,
    region=REGION,
    index_id=my_index.name,
    endpoint_id=my_index_endpoint.name,
    embedding=embedding_model,
    stream_update=True,
)

vector_store.add_texts(texts=texts
                       , is_complete_overwrite=True)

In [56]:
# Try running a simialarity search
vector_store.similarity_search("pizza")

[Document(page_content='eat pizza for'),
 Document(page_content='dinner.'),
 Document(page_content='the mat.'),
 Document(page_content='in the west.')]

### Create vectorstore with metadata filters

In [24]:
#Input text with metadata
record_data = [
    {
        "description": "A versatile pair of dark-wash denim jeans."
        "Made from durable cotton with a classic straight-leg cut, these jeans"
        " transition easily from casual days to dressier occasions.",
        "price": 65.00,
        "color": "blue",
        "season": ["fall", "winter", "spring"],
    },
    {
        "description": "A lightweight linen button-down shirt in a crisp white."
        " Perfect for keeping cool with breathable fabric and a relaxed fit.",
        "price": 34.99,
        "color": "white",
        "season": ["summer", "spring"],
    },
    {
        "description": "A soft, chunky knit sweater in a vibrant forest green. "
        "The oversized fit and cozy wool blend make this ideal for staying warm "
        "when the temperature drops.",
        "price": 89.99,
        "color": "green",
        "season": ["fall", "winter"],
    },
    {
        "description": "A classic crewneck t-shirt in a soft, heathered blue. "
        "Made from comfortable cotton jersey, this t-shirt is a wardrobe essential "
        "that works for every season.",
        "price": 19.99,
        "color": "blue",
        "season": ["fall", "winter", "summer", "spring"],
    },
    {
        "description": "A flowing midi-skirt in a delicate floral print. "
        "Lightweight and airy, this skirt adds a touch of feminine style "
        "to warmer days.",
        "price": 45.00,
        "color": "white",
        "season": ["spring", "summer"],
    },
]

In [25]:
# Parse and prepare input data

texts = []
metadatas = []
for record in record_data:
    record = record.copy()
    page_content = record.pop("description")
    texts.append(page_content)
    if isinstance(page_content, str):
        metadata = {**record}
        metadatas.append(metadata)

In [27]:
# NOTE : This operation can take upto 20 mins
vector_store = VectorSearchVectorStore.from_components(
    project_id=PROJECT_ID,
    region=REGION,
    gcs_bucket_name=BUCKET,
    index_id=my_index.name,
    endpoint_id=my_index_endpoint.name,
    embedding=embedding_model,
)

vector_store.add_texts(texts=texts, metadatas=metadatas, is_complete_overwrite=True)

Upserting datapoints MatchingEngineIndex index: projects/543079149601/locations/us-central1/indexes/6565496190752784384
MatchingEngineIndex index Upserted datapoints. Resource name: projects/543079149601/locations/us-central1/indexes/6565496190752784384


['dc9f3cc8-4e25-49a1-bb38-00d0f7476b8e',
 'bdf428e8-e9fc-4127-a856-6d81fc2cd295',
 'b29816f2-bfe7-42af-8fbd-f34b73c8564f',
 'c99e8e47-4bfc-4265-8931-2886b4fe15c4',
 '6153563d-f1b2-49c4-8b1c-3e7fcbcb8f2d']

In [28]:
from google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint import (
    Namespace,
    NumericNamespace,
)

In [29]:
# Try running a similarity search

# Below code should return 5 results
vector_store.similarity_search("shirt", k=5)

[Document(page_content='A lightweight linen button-down shirt in a crisp white. Perfect for keeping cool with breathable fabric and a relaxed fit.', metadata={'season': ['summer', 'spring'], 'color': 'white', 'price': 34.99}),
 Document(page_content='A classic crewneck t-shirt in a soft, heathered blue. Made from comfortable cotton jersey, this t-shirt is a wardrobe essential that works for every season.', metadata={'season': ['fall', 'winter', 'summer', 'spring'], 'color': 'blue', 'price': 19.99}),
 Document(page_content='A flowing midi-skirt in a delicate floral print. Lightweight and airy, this skirt adds a touch of feminine style to warmer days.', metadata={'season': ['spring', 'summer'], 'color': 'white', 'price': 45.0}),
 Document(page_content='A versatile pair of dark-wash denim jeans.Made from durable cotton with a classic straight-leg cut, these jeans transition easily from casual days to dressier occasions.', metadata={'season': ['fall', 'winter', 'spring'], 'color': 'blue', 

In [30]:
# Try running a similarity search with text filter
filters = [Namespace(name="season", allow_tokens=["spring"])]

# Below code should return 4 results now
vector_store.similarity_search("shirt", k=5, filter=filters)

[Document(page_content='A lightweight linen button-down shirt in a crisp white. Perfect for keeping cool with breathable fabric and a relaxed fit.', metadata={'season': ['summer', 'spring'], 'color': 'white', 'price': 34.99}),
 Document(page_content='A classic crewneck t-shirt in a soft, heathered blue. Made from comfortable cotton jersey, this t-shirt is a wardrobe essential that works for every season.', metadata={'season': ['fall', 'winter', 'summer', 'spring'], 'color': 'blue', 'price': 19.99}),
 Document(page_content='A flowing midi-skirt in a delicate floral print. Lightweight and airy, this skirt adds a touch of feminine style to warmer days.', metadata={'season': ['spring', 'summer'], 'color': 'white', 'price': 45.0}),
 Document(page_content='A versatile pair of dark-wash denim jeans.Made from durable cotton with a classic straight-leg cut, these jeans transition easily from casual days to dressier occasions.', metadata={'season': ['fall', 'winter', 'spring'], 'color': 'blue', 

In [31]:
# Try running a similarity search with combination of text and numeric filter
filters = [Namespace(name="season", allow_tokens=["spring"])]
numeric_filters = [NumericNamespace(name="price", value_float=40.0, op="LESS")]

# Below code should return 2 result now
vector_store.similarity_search(
    "shirt", k=5, filter=filters, numeric_filter=numeric_filters
)

[Document(page_content='A lightweight linen button-down shirt in a crisp white. Perfect for keeping cool with breathable fabric and a relaxed fit.', metadata={'season': ['summer', 'spring'], 'color': 'white', 'price': 34.99}),
 Document(page_content='A classic crewneck t-shirt in a soft, heathered blue. Made from comfortable cotton jersey, this t-shirt is a wardrobe essential that works for every season.', metadata={'season': ['fall', 'winter', 'summer', 'spring'], 'color': 'blue', 'price': 19.99})]